# 🎬 Processamento e Indexação de Filmes

Este notebook processa os arquivos de filmes, divide em chunks, gera embeddings e salva no PostgreSQL.

**Importante:** Verifica duplicatas antes de inserir!

## 1. Importações e Configurações

In [16]:
import os
import re
from pathlib import Path
from typing import List, Tuple
import psycopg2
from psycopg2.extras import execute_values
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import pandas as pd

print("✓ Bibliotecas importadas com sucesso")

✓ Bibliotecas importadas com sucesso


## 2. Configurações do Sistema

In [17]:
DB_CONFIG = {
    'dbname': 'filmes_rag',
    'user': 'postgres',
    'password': 'senha123',
    'host': 'localhost',
    'port': '5432'
}

DIRETORIO_FILMES = 'filmestxts'
MODELO_EMBEDDING = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
TAMANHO_CHUNK = 500
OVERLAP_CHUNK = 100

print("✓ Configurações carregadas")

✓ Configurações carregadas


## 3. Funções Auxiliares

In [18]:
def limpar_texto(texto: str) -> str:
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

def criar_chunks(texto: str, tamanho: int, overlap: int) -> List[str]:
    chunks = []
    inicio = 0
    while inicio < len(texto):
        fim = inicio + tamanho
        chunk = texto[inicio:fim]
        if len(chunk.strip()) > 50:
            chunks.append(chunk.strip())
        inicio += (tamanho - overlap)
    return chunks

def extrair_titulo(nome_arquivo: str) -> str:
    return Path(nome_arquivo).stem

print("✓ Funções auxiliares carregadas")

✓ Funções auxiliares carregadas


## 4. Conectar ao Banco de Dados

In [19]:
try:
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    print("✓ Conectado ao PostgreSQL")
    
    cursor.execute("SELECT extversion FROM pg_extension WHERE extname = 'vector';")
    versao = cursor.fetchone()
    if versao:
        print(f"✓ pgVector versão {versao[0]}")
    else:
        print("✗ pgVector NÃO encontrada!")
except Exception as e:
    print(f"✗ Erro: {e}")
    raise

✓ Conectado ao PostgreSQL
✓ pgVector versão 0.5.1


## 5. Funções de Banco de Dados

In [20]:
def verificar_filme_existe(titulo: str) -> bool:
    cursor.execute("SELECT COUNT(*) FROM filmes WHERE titulo = %s;", (titulo,))
    return cursor.fetchone()[0] > 0

def obter_chunks_existentes(titulo: str) -> set:
    cursor.execute("SELECT chunk_index FROM filmes WHERE titulo = %s;", (titulo,))
    return {row[0] for row in cursor.fetchall()}

def remover_filme(titulo: str):
    cursor.execute("DELETE FROM filmes WHERE titulo = %s;", (titulo,))
    conn.commit()

def inserir_chunks(titulo: str, chunks: List[str], embeddings: np.ndarray, indices_existentes: set = None):
    if indices_existentes is None:
        indices_existentes = set()
    
    dados_novos = [
        (titulo, chunk, idx, embedding.tolist())
        for idx, (chunk, embedding) in enumerate(zip(chunks, embeddings))
        if idx not in indices_existentes
    ]
    
    if not dados_novos:
        return 0
    
    query = """INSERT INTO filmes (titulo, chunk_texto, chunk_index, vetor_embedding)
               VALUES %s ON CONFLICT (titulo, chunk_index) DO NOTHING;"""
    execute_values(cursor, query, dados_novos)
    conn.commit()
    return len(dados_novos)

def listar_filmes_indexados() -> pd.DataFrame:
    query = """SELECT titulo, COUNT(*) as num_chunks, MIN(data_insercao) as data_insercao
               FROM filmes GROUP BY titulo ORDER BY titulo;"""
    return pd.read_sql_query(query, conn)

print("✓ Funções de BD carregadas")

✓ Funções de BD carregadas


## 6. Carregar Modelo

In [21]:
print("Carregando modelo...")
modelo = SentenceTransformer(MODELO_EMBEDDING)

if modelo.device.type == 'cuda':
    import torch
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("✓ CPU (mais lento)")

print(f"✓ Dimensão: {modelo.get_sentence_embedding_dimension()}")

Carregando modelo...
✓ GPU: NVIDIA GeForce RTX 3050 6GB Laptop GPU
✓ Dimensão: 768


## 7. Ver Filmes Indexados

In [22]:
df_indexados = listar_filmes_indexados()
if len(df_indexados) > 0:
    print(f"📊 {len(df_indexados)} filmes indexados\n")
    display(df_indexados)
else:
    print("📊 Nenhum filme indexado")

📊 Nenhum filme indexado


C:\Users\Lucas Fritzke\AppData\Local\Temp\ipykernel_3900\3491706269.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


## 8. Listar Arquivos

In [23]:
caminho_filmes = Path(DIRETORIO_FILMES)
arquivos_txt = list(caminho_filmes.glob("*.txt")) if caminho_filmes.exists() else []

print(f"📁 {len(arquivos_txt)} arquivos em '{DIRETORIO_FILMES}'\n")

if arquivos_txt:
    filmes_info = []
    for arquivo in arquivos_txt:
        titulo = extrair_titulo(arquivo.name)
        existe = verificar_filme_existe(titulo)
        filmes_info.append({
            'Arquivo': arquivo.name,
            'Título': titulo,
            'Tamanho (KB)': f"{arquivo.stat().st_size / 1024:.1f}",
            'Status': '✓ Indexado' if existe else '⏳ Pendente'
        })
    
    df_arquivos = pd.DataFrame(filmes_info)
    display(df_arquivos)
    
    pendentes = len([f for f in filmes_info if '⏳' in f['Status']])
    print(f"\n{len(arquivos_txt) - pendentes} indexados, {pendentes} pendentes")

📁 126 arquivos em 'filmestxts'



AttributeError: module 'numpy' has no attribute 'void'

AttributeError: module 'numpy' has no attribute 'void'


0 indexados, 126 pendentes


## 9. Processar Novos Filmes

In [24]:
print("🚀 Processando novos filmes...\n")

stats = {'processados': 0, 'pulados': 0, 'chunks': 0, 'erros': []}

for arquivo in tqdm(arquivos_txt, desc="Processando"):
    try:
        titulo = extrair_titulo(arquivo.name)
        
        if verificar_filme_existe(titulo):
            print(f"⏭️ {titulo}")
            stats['pulados'] += 1
            continue
        
        print(f"\n📄 {titulo}")
        
        with open(arquivo, 'r', encoding='utf-8') as f:
            conteudo = f.read()
        
        texto_limpo = limpar_texto(conteudo)
        chunks = criar_chunks(texto_limpo, TAMANHO_CHUNK, OVERLAP_CHUNK)
        print(f"   {len(chunks)} chunks")
        
        if not chunks:
            continue
        
        embeddings = modelo.encode(chunks, batch_size=32, show_progress_bar=False, convert_to_numpy=True)
        inseridos = inserir_chunks(titulo, chunks, embeddings)
        print(f"   ✓ {inseridos} inseridos")
        
        stats['processados'] += 1
        stats['chunks'] += inseridos
        
    except Exception as e:
        print(f"   ✗ Erro: {e}")
        stats['erros'].append(f"{arquivo.name}: {e}")

print("\n" + "="*60)
print("📊 RESUMO")
print("="*60)
print(f"Processados: {stats['processados']}")
print(f"Pulados: {stats['pulados']}")
print(f"Chunks: {stats['chunks']}")
print(f"Erros: {len(stats['erros'])}")
if stats['erros']:
    for erro in stats['erros']:
        print(f"  - {erro}")

🚀 Processando novos filmes...



Processando:   0%|          | 0/126 [00:00<?, ?it/s]


📄 007 - Operação Skyfall
   15 chunks
   ✓ 15 inseridos

📄 1917
   10 chunks
   ✓ 10 inseridos

📄 A Bela e a Fera
   7 chunks
   ✓ 7 inseridos

📄 A Caça
   9 chunks
   ✓ 9 inseridos

📄 A Era do Gelo 3
   9 chunks
   ✓ 9 inseridos

📄 A Forma da Água
   9 chunks
   ✓ 9 inseridos

📄 A Grande Aposta
   12 chunks
   ✓ 12 inseridos

📄 A Grande Beleza
   10 chunks
   ✓ 10 inseridos

📄 A Hora Mais Escura
   12 chunks
   ✓ 12 inseridos

📄 A Invenção de Hugo Cabret
   15 chunks
   ✓ 15 inseridos

📄 A Juventude
   12 chunks
   ✓ 12 inseridos

📄 A Origem
   8 chunks
   ✓ 8 inseridos

📄 A Separação
   10 chunks
   ✓ 10 inseridos

📄 A Substância
   16 chunks
   ✓ 16 inseridos

📄 A Teoria de Tudo
   9 chunks
   ✓ 9 inseridos

📄 A Árvore da Vida
   13 chunks
   ✓ 13 inseridos

📄 Ainda Estou Aqui
   16 chunks
   ✓ 16 inseridos

📄 Anomalisa
   7 chunks
   ✓ 7 inseridos

📄 Appaloosa - Uma Cidade Sem Lei
   7 chunks
   ✓ 7 inseridos

📄 Aquarius
   11 chunks
   ✓ 11 inseridos

📄 Artista do Desastre
   9 c

## 10. Reprocessar Filme Específico

In [ ]:
ARQUIVO_REPROCESSAR = "007 - Operação Skyfall.txt"

arquivo_path = caminho_filmes / ARQUIVO_REPROCESSAR

if arquivo_path.exists():
    titulo = extrair_titulo(ARQUIVO_REPROCESSAR)
    
    if verificar_filme_existe(titulo):
        print(f"🗑️ Removendo '{titulo}'...")
        remover_filme(titulo)
        print("✓ Removido\n")
    
    print(f"📄 Reprocessando: {titulo}\n")
    
    with open(arquivo_path, 'r', encoding='utf-8') as f:
        conteudo = f.read()
    
    texto_limpo = limpar_texto(conteudo)
    chunks = criar_chunks(texto_limpo, TAMANHO_CHUNK, OVERLAP_CHUNK)
    embeddings = modelo.encode(chunks, show_progress_bar=True, convert_to_numpy=True)
    inseridos = inserir_chunks(titulo, chunks, embeddings)
    
    print(f"\n✓ {inseridos} chunks inseridos")
else:
    print(f"✗ Arquivo não encontrado")

## 11. Limpar Tudo (CUIDADO!)

In [15]:
# Descomente para usar:
resposta = input("⚠️ APAGAR TUDO? (digite SIM): ")
if resposta == "SIM":
    cursor.execute("TRUNCATE TABLE filmes RESTART IDENTITY;")
    conn.commit()
    print("✓ Banco limpo")
else:
    print("❌ Cancelado")

print("ℹ️ Célula comentada por segurança")

✓ Banco limpo
ℹ️ Célula comentada por segurança


## 12. Estatísticas Finais

In [25]:
print("\n📊 ESTATÍSTICAS")
print("="*60)

cursor.execute("SELECT COUNT(*) FROM filmes;")
total_chunks = cursor.fetchone()[0]
print(f"Chunks: {total_chunks}")

cursor.execute("SELECT COUNT(DISTINCT titulo) FROM filmes;")
total_filmes = cursor.fetchone()[0]
print(f"Filmes: {total_filmes}")

if total_filmes > 0:
    print(f"Média: {total_chunks / total_filmes:.1f} chunks/filme")

cursor.execute("SELECT pg_size_pretty(pg_database_size('filmes_rag'));")
print(f"Tamanho: {cursor.fetchone()[0]}")

print("\n")
df_final = listar_filmes_indexados()
if len(df_final) > 0:
    display(df_final)


📊 ESTATÍSTICAS
Chunks: 1319
Filmes: 126
Média: 10.5 chunks/filme
Tamanho: 20 MB




C:\Users\Lucas Fritzke\AppData\Local\Temp\ipykernel_3900\3491706269.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


AttributeError: module 'numpy' has no attribute 'void'

AttributeError: module 'numpy' has no attribute 'void'

## 13. Testar Busca

In [26]:
def buscar_teste(query: str, top_k: int = 3):
    print(f"\n🔍 '{query}'\n")
    
    emb = modelo.encode([query], convert_to_numpy=True)[0]
    
    sql = """SELECT titulo, chunk_texto, 1 - (vetor_embedding <=> %s::vector) as sim
             FROM filmes ORDER BY vetor_embedding <=> %s::vector LIMIT %s;"""
    
    cursor.execute(sql, (emb.tolist(), emb.tolist(), top_k))
    
    for i, (titulo, chunk, sim) in enumerate(cursor.fetchall(), 1):
        print(f"[{i}] {titulo} ({sim:.1%})")
        print(f"    {chunk[:120]}...\n")

buscar_teste("filme de ação")
buscar_teste("algum filme de comédia romântica")


🔍 'filme de ação'

[1] La La Land - Cantando Estações (48.6%)
    do Teatro Dolby, em Los Angeles, caso ela, pelo menos, não seja indicada ao Oscar).Nesse ponto, caberia a pergunta: “Ué,...

[2] O Som ao Redor (43.5%)
    partir da criação da MTV, nos deparamos com produções cada vez mais barulhentas e equivocadas do ponto de vista sonoro. ...

[3] Tick, Tick…Boom! (33.9%)
    sobre os dilemas de ser artista, que acaba tocando em outras questões filosóficas. Logo, para falar disso, surge um outr...


🔍 'algum filme de comédia romântica'

[1] Coração Louco (57.5%)
    nda com o auxílio luxuoso de Robert Duvall e do menino Jack Nation, que dá vida a um maravilhoso Buddy. Assim, o filme d...

[2] O Artista (55.8%)
    o. Na verdade, sua sutileza e simplicidade o fazem interessante e, talvez, seja essa a razão do encantamento que tem ger...

[3] O Farol (55.1%)
    dois homens oprimidos, impossibilitados de pegar um barco e partir da ilha. A convivência forçada entre os amigos/inimig...



## 14. Fechar Conexão

In [17]:
cursor.close()
conn.close()
print("✓ Conexão fechada")
print("\n✅ Pronto!")
print("   Execute: streamlit run interface_busca.py")

✓ Conexão fechada

✅ Pronto!
   Execute: streamlit run interface_busca.py
